In [1]:
import pandas as pd
import numpy as np
from personal_utils import *

In [5]:
# Load single ts
data_path = 'data/AppliancesEnergy_TRAIN.ts'
ts_1 = load_single_ts(data_path, 5, 3)
matrix_appliances = load_datapoint(data_path, 0)

119it [00:02, 48.78it/s]
100%|██████████| 95/95 [00:00<00:00, 871.90it/s]
119it [00:02, 49.85it/s]
100%|██████████| 95/95 [00:00<00:00, 866.78it/s]


In [6]:
data_path2 = 'data/IEEEPPG_TEST.ts'
matrix_ieee = load_datapoint(data_path2, 100)
ts_ie = load_single_ts(data_path2, 5, 3)

1351it [00:01, 769.08it/s]
100%|██████████| 1328/1328 [00:00<00:00, 2684.73it/s]
1351it [00:01, 782.64it/s]
100%|██████████| 1328/1328 [00:00<00:00, 3081.45it/s]


In [47]:
#Test general shape of compression
dr = 0.4
print(ts_ie[:100])
print(dwt_compress(ts_ie[0:100], dr ))
#print(compute_rmse(ts_ie, dwt_compress(ts_ie, dr )))

[-0.3276 -0.3588 -0.351  -0.3666 -0.312  -0.3276 -0.3354 -0.351  -0.3354
 -0.3276 -0.312  -0.3354 -0.3744 -0.3354 -0.3354 -0.3666 -0.351  -0.351
 -0.3276 -0.351  -0.3198 -0.3276 -0.312  -0.3354 -0.273  -0.3198 -0.3354
 -0.3354 -0.3042 -0.312  -0.2652 -0.2886 -0.3198 -0.2964 -0.3198 -0.3198
 -0.3198 -0.3354 -0.351  -0.3822 -0.3666 -0.351  -0.351  -0.3276 -0.3666
 -0.351  -0.3432 -0.3432 -0.3432 -0.3588 -0.312  -0.3354 -0.3276 -0.2886
 -0.3198 -0.2886 -0.3354 -0.3042 -0.3042 -0.312  -0.3198 -0.312  -0.3354
 -0.3198 -0.3276 -0.3198 -0.3198 -0.2964 -0.3276 -0.3354 -0.3042 -0.3354
 -0.3432 -0.3198 -0.3432 -0.3588 -0.3432 -0.3276 -0.3432 -0.351  -0.3666
 -0.3432 -0.3588 -0.312  -0.3276 -0.312  -0.3354 -0.3198 -0.3198 -0.3198
 -0.2886 -0.3042 -0.2886 -0.312  -0.3354 -0.312  -0.3588 -0.2808 -0.3042
 -0.3198]
Signal length: 100, Wavelet: db4, Max level: 3
Decomposition level: 3
Initial wavelet coefficients: [array([-0.97190121, -0.96817342, -0.97574479, -0.96734429, -0.97937073,
       -0.94350

In [36]:
# Compute dft on every column of matrix
matrix = matrix_ieee

matrix_compressed = np.zeros(matrix.shape)
for i in range(matrix.shape[1]):
    matrix_compressed[:,i] = dwt_compress(matrix[:,i], 0.65)


compute_rmse_of_datapoint(matrix, matrix_compressed)

#(1.4435339091230222,
#array([3.71996501, 3.38425485, 0.03060634, 0.0288563 , 0.05398704]))

(0.9745457315927168,
 array([2.51989208, 2.29014343, 0.01868709, 0.01885416, 0.02515189]))

Testing calculateCompRatio

In [ ]:
# Testing calculateCompRatio
import gzip

def calculateCompRatio(dataset_array, array_flatdim_comp):

    # Calculate the number of bytes in the raw dataset array
    num_bytes_raw = dataset_array.nbytes
    print("Number of bytes in the raw dataset array:", num_bytes_raw)
    print("Number of the bytes in array_comp before gzip", array_flatdim_comp.nbytes)
    # Flatten the compressed array in column-major order. # Tested it, not need to put .copy() after.
    array_flatdim_1d = array_flatdim_comp.reshape(-1, order='F')
    print("Array flattened", array_flatdim_1d)

    # Convert the flattened array to a byte string
    compressed_data_bytestring = array_flatdim_1d.tobytes()

    print("Size of raw_data after gzip", len(gzip.compress(dataset_array.tobytes())))

    # Compress the byte string using gzip compression
    compressed_data_gzipd = gzip.compress(compressed_data_bytestring, compresslevel=9)
    print("Size of the compressed data after gzip:", len(compressed_data_gzipd))



    # Calculate and return the compression ratio
    compression_ratio = num_bytes_raw / len(compressed_data_gzipd)
    print("Compression ratio:", compression_ratio)
    return compression_ratio


In [ ]:
# Try with easy data. Then try with real time-series.

num_dp = 3
len_ts = 9
num_dim = 4

array = np.arange(num_dp * len_ts * num_dim)
array_comp = np.arange(num_dp * len_ts * num_dim)



array_comp = array_comp.reshape(num_dp * len_ts, num_dim)

array_comp[[1,2,3,5,6,7,9,12,13,14,15,16,15,16,17,20]] = 0


print(array_comp)
calculateCompRatio(array, array_comp)


# Conclusions:
# Values look good! Show that also with zeroing out we reached a little more compression!